In [8]:
from __future__ import division, print_function # Imports from __future__ since we're running Python 2
import os
import numpy as np
import collections as cls
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import re
import itertools
import math
import glob
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.linear_model import RandomizedLasso
from sklearn.neural_network import MLPRegressor
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from sklearn.metrics import mean_absolute_error
from matplotlib import pylab
from pylab import *
random_state = 10

In [9]:
#get all city names
all_cities_paths =  glob.glob(os.path.join(os.getcwd(), 'data','*'))
cities = []
for city in all_cities_paths:
    cities.append(city.split("/")[-1])
print(cities)
                    

['amsterdam', 'berlin', 'copenhagen', 'dublin', 'edinburgh', 'helsinki', 'lisbon', 'london', 'madrid', 'moscow', 'rome', 'tallin', 'vienna', 'paris', 'reykjavik']


In [10]:
#finds the not common elements of the two lists
#input: list1,list2 --> the lists to be compared
#output: the not common elements
def find_not_common_columns(list1,list2):
    not_common_cols = []
    both_lists = list1 + list2
    for col in set(both_lists):
        if col not in set(list1) or col not in set(list2):
            not_common_cols.append(col)
    return not_common_cols



In [11]:
#loading each city's dataset to a dictionary and stripping their column titles from unwanted characters
#moreover checking which columns are not common for all csv files of a particular city in order to be discarded later
#in addition their last_modified column values are stripped
datasets = dict()
for city in cities:
    city_data = pd.DataFrame() #initializing city's dataframe
    data_path = glob.glob(os.path.join(os.getcwd(), 'data',city,city,city,'*.csv'))
    print('\n\n\n{}\n'.format(city))
    first = True
    file_columns = [] #containing the columns that the first file of the particular city contains
    for f in data_path:
        file_data =pd.read_csv(f,delimiter=",",lineterminator="\n") 
        old_columns = list(file_data.columns.values)
        new_columns = [] #containing the columns after the stripping
        for col in old_columns:
            new_columns.append(col.strip())
        if first:
            file_columns = new_columns
            first = False
        else:
            if file_columns == new_columns:
                print('no difference in the columns')
            else:
                print('DIFFERENCE IN THE COLUMNS REPORTED')
                print('NOT COMMON COLUMNS: {}'.format(find_not_common_columns(file_columns,new_columns)))      
        file_data.columns = new_columns
        file_data['last_modified'] = file_data['last_modified'].apply(lambda x: x.strip()) #stripping all the values of 'last_modified' column
        city_data = pd.concat([city_data,file_data],ignore_index=True)
    print('\n{}\'s dataset length: {}'.format(city,len(city_data.index)))
    #city_data.fillna("(unknown)",inplace=True)
    datasets[city] = city_data




amsterdam

no difference in the columns
no difference in the columns
no difference in the columns
no difference in the columns
no difference in the columns
no difference in the columns
no difference in the columns
no difference in the columns
no difference in the columns
no difference in the columns
no difference in the columns
DIFFERENCE IN THE COLUMNS REPORTED
NOT COMMON COLUMNS: ['survey_id', 'bathrooms', 'city', 'country', 'location', 'name']
DIFFERENCE IN THE COLUMNS REPORTED
NOT COMMON COLUMNS: ['survey_id', 'bathrooms', 'city', 'country', 'location', 'name']

amsterdam's dataset length: 184551



berlin

no difference in the columns
no difference in the columns
no difference in the columns
no difference in the columns
no difference in the columns
no difference in the columns
no difference in the columns
no difference in the columns
no difference in the columns
no difference in the columns
no difference in the columns
no difference in the columns
no difference in the columns
D

In [12]:
#find out which of the columns of each city's dataset have missing values 
#and delete the columns that have only missing values
city_columns = dict() #it is going to conatin the feature list of each city
for city in datasets.keys():
    city_data = datasets[city]
    all_data = len(city_data.index)
    columns = city_data.columns.values
    city_columns[city] = list(columns)
    some_mis = dict()
    only_mis = []
    for col in columns:
        #bools = data[col].to_frame().isin({col: ['(unknown)']}) 
        bools = city_data[col].to_frame().isnull()
        num = len(bools[bools[col] == True].index) #number of mssing values in the particular column
        if num > 0: #if there is at least one missing value
            if num < len(city_data[col].index): #if not all values are missing
                some_mis[col] = num
            else:
                only_mis.append(col) #if all values of the column are missing
    print_str = '\n{} '.format(city)
    if len(some_mis.keys()) > 0:
        first = True
        for col in some_mis.keys():
            if first:
                first = False
                print_str += '\ncols with some missing values--> {}({}%)'.format(col,100*float(some_mis[col]/all_data))
            else:
                print_str += ',{}({}%)'.format(col,100*float(some_mis[col]/all_data))
    if len(only_mis) > 0:
        print_str += '\ncols with only missing values--> {}'.format(only_mis[0])
        for i in range(1,len(only_mis)):
            print_str += ',{}'.format(only_mis[i])
        city_data.drop(only_mis, axis=1, inplace=True) #deleting columns with only missing data
        datasets[city] = city_data
    print(print_str)
        
    


edinburgh 
cols with some missing values--> overall_satisfaction(13.393982944%),city(69.7384655016%),name(69.7411771086%),price(0.00271160703391%),bedrooms(0.139647762246%),accommodates(1.75712135797%),minstay(72.1260354949%),location(69.7384655016%),survey_id(69.7384655016%),room_type(0.00677901758477%)
cols with only missing values--> bathrooms,borough,country

reykjavik 
cols with some missing values--> city(78.223078474%),price(0.0201617419745%),bedrooms(9.57010685723%),overall_satisfaction(16.4363000963%),minstay(57.7745917247%),location(78.223078474%),survey_id(78.223078474%),room_type(0.0201617419745%)
cols with only missing values--> bathrooms,borough,country

helsinki 
cols with some missing values--> city(64.1407968712%),survey_id(64.1407968712%),name(64.3241261305%),bedrooms(0.0427768271816%),overall_satisfaction(12.1425079443%),minstay(88.1935956979%),location(64.1407968712%),property_type(64.1407968712%)
cols with only missing values--> bathrooms,borough,country

paris 
c

In [ ]:
df = pd.DataFrame([[1, 2, 2, 0], [2, np.nan, 3, 1], [3, 4, 3, 1], [2, 4, 3, 1], [2, 4, 3, 1],
                   [3, np.nan, 3, 1], [3, 4, 3, 1]],columns=list('ABCD'))
col='B'
null_entries = df[['A']][df['B'].isnull()]
for index,row in null_entries.iterrows():
    same_room_entries = df[[col]][df['A'] == row['A']] #get the rest of the 
    if len(same_room_entries.index) > 0:
        df.set_value(index,col,same_room_entries[col].median())

df['B'][0]

2.0

In [ ]:
#dealing with the columns which contain both missing and normal entries
columns_to_delete = ['city','location','survey_id','name','minstay','property_type']
#city is not needed
#location is not needed
#survey_id is not needed
#name is not needed
#minstay have too many null values
#overall satisfaction cannot be used both because of the many null entries and because the reviews 
#are usually not enough to produce reliable assumptions
#property_type has too many values

delete_null_entries = ['price']
#these columns' values are too significant and they might change from time to time even for the same room
#as a result it seems wiser to delete the entries where these columns are null rather than replace them
#accommodates will be dealt with later

replace_nulls_median = ['bedrooms','overall_satisfaction']
#given the fact that generally the same room appears multiple times inside each city's dataset these columns are going 
#to be replaced by the median of the column values inside the same room through time

replace_nulls_value = ['room_type','host_id']
#if the same room_id exists in the dataset with a non-null value for these fields then the null value is replaced 
#by that. If not the entries containing null values are removed

for city in datasets.keys():
    print('\n{}'.format(city))
    city_data = datasets[city]
    for col in columns_to_delete:
        if col in city_data.columns:
            city_data.drop([col], axis=1, inplace=True) #columns_to_delete are deleted
    old_num = len(city_data.index)
    
    for col in delete_null_entries:
        if col in city_data.columns:
            city_data.dropna(axis=0,subset = [col],inplace=True) #the null entries of delete_null_entries are deleted
        print('\n{}:\t{} values were deleted'.format(
                col,len(city_data[city_data[col].isnull()].index)))
            
    for col in replace_nulls_median:
        if col in city_data.columns:
            null_entries = city_data[['room_id']][city_data[col].isnull()]
            count = 0
            for index,row in null_entries.iterrows():
                same_room_entries = city_data[[col]][city_data['room_id'] == row['room_id']] 
                #get the rest of the entries of the particular room where the nul appeared
                if len(same_room_entries.index) > 0:
                    city_data.set_value(index,col,same_room_entries[col].median())
                    count += 1
                    #replacing the null value with the median of the values for the same room_id
            print('\n{}:\t{} values were replaced\t{} were not replaced and were deleted'.format(
                col,count,len(city_data[city_data[col].isnull()].index)))
            
            city_data.dropna(axis=0,subset = [col],inplace=True) #the null entries that were not replaced are deleted
    
    for col in replace_nulls_value:
        if col in city_data.columns:
            null_entries = city_data[['room_id']][city_data[col].isnull()]
            count = 0
            for index,row in null_entries.iterrows():
                same_room_entries = city_data[col][city_data['room_id'] == row['room_id']] 
                #get the rest of the entries of the particular room where the null appeared
                if len(same_room_entries.index) > 0:
                    city_data.set_value(index,col,same_room_entries.values[0])
                    count += 1
                    #replacing the null value with the value existing in the 
                    #same column of the first entry with the same room_id that appears
            print('\n{}:\t{} values were replaced\t{} were not replaced and were deleted'.format(
                col,count,len(city_data[city_data[col].isnull()].index)))
            
            city_data.dropna(axis=0,subset = [col],inplace=True) #the null entries that were not replaced are deleted    
            
            
    new_num = len(city_data.index)
    print('\n{}% of the {}\'s dataset was deleted'.format(100 * float((old_num-new_num)/len(city_data.index)), city))
    datasets[city] = city_data




edinburgh

price:	0 values were deleted

bedrooms:	101 values were replaced	17 were not replaced and were deleted

overall_satisfaction:	9872 values were replaced	1845 were not replaced and were deleted

room_type:	3 values were replaced	1 were not replaced and were deleted

host_id:	0 values were replaced	0 were not replaced and were deleted

2.59416903132% of the edinburgh's dataset was deleted

reykjavik

price:	0 values were deleted

bedrooms:	4263 values were replaced	23 were not replaced and were deleted

overall_satisfaction:	7315 values were replaced	922 were not replaced and were deleted

room_type:	0 values were replaced	0 were not replaced and were deleted

host_id:	0 values were replaced	0 were not replaced and were deleted

2.18381595513% of the reykjavik's dataset was deleted

helsinki

price:	0 values were deleted

bedrooms:	7 values were replaced	3 were not replaced and were deleted

overall_satisfaction:	1984 values were replaced	230 were not replaced and were deleted

In [ ]:
#datasets['vienna'][datasets['vienna']['overall_satisfaction'] == 5835467]

In [ ]:
#find out which columns are existent to all of the cities and which are not
all_columns = set() # the union of columns appearing in all of the cities
all_common_columns = set() # the intersection of columns appearing in all of the cities
first = True
for city,columns in city_columns.iteritems():
    columns_set = set(columns) #since all the columns of the same city have a unique name
    for col in columns:
        all_columns.add(col)
    if first == False:
        mortals = [] #containing the columns that are going to be removed from the intersection
        for col in all_common_columns:
            if col not in columns_set:
                mortals.append(col)
        for mor in mortals: 
            all_common_columns.remove(mor)
    else:
        for col in columns:
            all_common_columns.add(col) #initializing the intersection
        first = False

all_common_list = list(all_common_columns) #set to list
if len(all_common_list) > 0:
    print_str = 'Features appearing to every city of the dataset:\n{}'.format(all_common_list[0])
    for i in range(1,len(all_common_list)): #the rest
        print_str += ',{}'.format(all_common_list[i])
    print(print_str)
else:
    print("There are no common features between the different cities")


print_str = "\nFeatures appearing only to some cities of the dataset:\n"
first = True
for col in all_columns:
    if(col not in all_common_columns):
        if first == True:
            print_str += col
            first = False
        else:
            print_str += "," + col

print(print_str)


    

In [ ]:
df = pd.DataFrame([[np.nan, 2, np.nan, 0], [3, 4, np.nan, 1]],columns=list('ABCD'))
df
#df.dropna(axis=0,subset=['A'])

In [ ]:
#checking if some bad values exist in accommodates column for every city
bad_accommodates_value_found = False
error_cities = [] #containing the cities containing bad values in that column
for city,data in datasets.iteritems():
    print('\n{}:\n'.format(city))
    error_found = False
    bools = data['accommodates'].to_frame().isnull()
    numeric_error_entries = len(data[data['accommodates'] < 1].index) #number of entries with values less than 1
    missing_entries = len(bools[bools['accommodates'] == True].index) #number of entries with missing values
    if numeric_error_entries > 0:
        error_found = True
        print('Numeric error for {} entries ({}% of the city\'s dataset)'.format(numeric_error_entries,
                                                                                100*float(numeric_error_entries/
                                                                                         len(data.index))))
    if missing_entries > 0:
        error_found = True
        print('Missing value for {} entries ({}% of the city\'s dataset)'.format(missing_entries,
                                                                                100*float(missing_entries/
                                                                                         len(data.index))))
    if error_found:
        error_cities.append(city)
        bad_accommodates_value_found = True    

    
#deleting the bad value entries:
print('\n\n')
for city in error_cities:
    city_data = datasets[city]
    old_num = len(city_data.index)
    city_data = city_data[city_data['accommodates'] >= 1]
    city_data = city_data.dropna(axis=0,subset = ['accommodates'])
    new_num = len(city_data.index)
    datasets[city] = city_data
    print('{} entries were deleted from {}'.format(old_num - new_num,city))



In [ ]:
#create new column: price per number of people that can be accommodated
for city in datasets.keys():
    city_data = datasets[city]
    city_data['price_per_accommodated'] = city_data['price'] / city_data['accommodates']
    datasets[city] = city_data

In [ ]:
#create new column: price per number of people that can be accommodated


In [ ]:
#input: p--> price, quartiles --> quartiles of price_per|_accommodated column,
#k --> tukey test's hyperparameter
#output: whether p is a low outlier, a high outlier or a normal instance
def tukey_test(p,quartiles,k):
    iqr = quartiles[2] - quartiles[0]
    lower_bound = quartiles[0] - k * iqr
    if(lower_bound < 1):
        lower_bound = 1 #we will not use the lower bound of tukey test but a fixed positive lower bound
        
    upper_bound  = quartiles[2] + k * iqr
    low_outliers = 0
    high_outliers = 0
    if p < lower_bound:
        return "low_outlier"
    elif p > upper_bound:
        return "high_outlier"
    return "no" 

In [ ]:
#checking if 'last_modified' alone can be used as the primary key
for city,data in datasets.iteritems():
    print(city)
    gb = data.groupby(['last_modified'])
    print(gb['last_modified'].count().sort_values(ascending=False)[0])
#for some cities it cannot be used

In [ ]:
#checking if 'last_modified' along with 'room_id' can be used as the primary key
for city,data in datasets.iteritems():
    print(city)
    print(data.groupby(['last_modified','room_id']).size().sort_values(ascending=False)[0])
#they can be used

In [ ]:
#given the fact that some prices might be per month outlier detection method tukey test will be used
#so that these entries will be captures as outliers along with the rest of the outliers the dataset might have
#price_per_accommodated will be used instead of price column

k = 1.5 #tukey test's hyperparameter
all_low_outliers = 0
all_high_outliers = 0
all_data = 0
print('OUTLIER DETECTION AND DELETION')

for city in datasets.keys():
    mortal_indexes = []
    city_data = datasets[city]
    quartiles = []
    quartiles.append(city_data['price_per_accommodated'].to_frame().quantile(q=0.25,axis=0)[0])
    quartiles.append(city_data['price_per_accommodated'].to_frame().mean(axis=0)[0])
    quartiles.append(city_data['price_per_accommodated'].to_frame().quantile(q=0.75,axis=0)[0])
    city_low_outliers = 0
    city_high_outliers = 0
    city_data.reset_index(inplace=True)
    for index,row in city_data.iterrows():
        res = tukey_test(row['price_per_accommodated'],quartiles,k)
        if res == 'low_outlier':
            city_low_outliers += 1
            mortal_indexes.append(index)
            #mortal_room_id.append(row['room_id'])
            #mortal_last_modified.append(row['last_modified'])
        elif res == 'high_outlier':
            city_high_outliers += 1
            mortal_indexes.append(index)
            #mortal_room_id.append(row['room_id'])
            #mortal_last_modified.append(row['last_modified'])
    all_data += len(city_data.index)
    all_low_outliers += city_low_outliers
    all_high_outliers += city_high_outliers
    city_data = city_data.drop(city_data.index[mortal_indexes]) #deleting the outliers by their index
    
    datasets[city] = city_data
    print('{0}\t(LOW)\t{1}\t(HIGH)\t{2}\t(TOTAL)\t{3}\t{4}% of city\'s dataset'.
          format(city,city_low_outliers,city_high_outliers,city_low_outliers + city_high_outliers
                 ,100 * float((city_low_outliers + city_high_outliers)/len(city_data.index))))

print('\n\n\nALTOGETHER\t(LOW)\t{0}\t(HIGH)\t{1}\t(TOTAL)\t{2}\t{3}% of all cities\' datasets'.
          format(all_low_outliers,all_high_outliers,all_low_outliers + all_high_outliers
                 ,100 * float((all_low_outliers + all_high_outliers)/all_data)))
            
            
    
    

In [ ]:
#checking if 'last_modified' alone can be used as the primary key
for city,data in datasets.iteritems():
    print(city)
    gb = data.groupby(['room_type'])
    print(gb['room_type'].count().sort_values(ascending=False))


In [ ]:
#creating bar chart showing the average prices and average prices per accommodated for all room types, per city
plt.clf()
N = len(datasets.keys())
price_means = []
price_stds = []
price_per_accommodated_means = []
price_per_accommodated_stds = []
cities = []

for city in datasets.keys():
    city_data = datasets[city]
    price_means.append(city_data['price'].mean())
    price_stds.append(city_data['price'].std())
    price_per_accommodated_means.append(city_data['price_per_accommodated'].mean())
    price_per_accommodated_stds.append(city_data['price_per_accommodated'].std())
    cities.append(city)


sort_indices = np.argsort(np.asarray(price_per_accommodated_means)) #sort cities on price per accommodated
price_means = list(np.asarray(price_means)[sort_indices[::-1]])
price_stds = list(np.asarray(price_stds)[sort_indices[::-1]])
price_per_accommodated_means = list(np.asarray(price_per_accommodated_means)[sort_indices[::-1]])
price_per_accommodated_stds = list(np.asarray(price_per_accommodated_stds)[sort_indices[::-1]])
cities = list(np.asarray(cities)[sort_indices[::-1]])


pos = np.arange(N)
width = 0.35

fig, ax = plt.subplots(figsize=(20,3))

prices_bar = ax.bar(pos, tuple(price_means), width, color='b', yerr=tuple(price_stds))
prices_pa_bar = ax.bar(pos+width, tuple(price_per_accommodated_means), width, 
                       color='g', yerr=tuple(price_per_accommodated_stds))

ax.legend((prices_bar[0], prices_pa_bar[0]), ('Price', 'Price per Accommodated'))

ax.set_ylabel('Dollars($)')
ax.set_title('Average price and price per person, per city')
ax.set_xticks(pos + width / 2)
ax.set_xticklabels(tuple(cities))

plt.show()

In [ ]:
#creating bar chart showing the average prices and average prices per accommodated for all room types, per city
plt.clf()
N = len(datasets.keys())
price_means = []
price_stds = []
price_per_accommodated_means = []
price_per_accommodated_stds = []
cities = []

for city in datasets.keys():
    city_data = datasets[city]
    city_data = city_data[city_data['room_type'] == 'Entire home/apt']
    price_means.append(city_data['price'].mean())
    price_stds.append(city_data['price'].std())
    price_per_accommodated_means.append(city_data['price_per_accommodated'].mean())
    price_per_accommodated_stds.append(city_data['price_per_accommodated'].std())
    cities.append(city)


sort_indices = np.argsort(np.asarray(price_per_accommodated_means)) #sort cities on price per accommodated
price_means = list(np.asarray(price_means)[sort_indices[::-1]])
price_stds = list(np.asarray(price_stds)[sort_indices[::-1]])
price_per_accommodated_means = list(np.asarray(price_per_accommodated_means)[sort_indices[::-1]])
price_per_accommodated_stds = list(np.asarray(price_per_accommodated_stds)[sort_indices[::-1]])
cities = list(np.asarray(cities)[sort_indices[::-1]])


pos = np.arange(N)
width = 0.35

fig, ax = plt.subplots(figsize=(20,3))

prices_bar = ax.bar(pos, tuple(price_means), width, color='b', yerr=tuple(price_stds))
prices_pa_bar = ax.bar(pos+width, tuple(price_per_accommodated_means), width, 
                       color='g', yerr=tuple(price_per_accommodated_stds))

ax.legend((prices_bar[0], prices_pa_bar[0]), ('Price', 'Price per Accommodated'))

ax.set_ylabel('Dollars($)')
ax.set_title('Average Price and price per person for \'entire home/apt\' room type, per city')
ax.set_xticks(pos + width / 2)
ax.set_xticklabels(tuple(cities))

plt.show()


In [ ]:
#creating bar chart showing the average prices and average prices per accommodated for all room types, per city
plt.clf()
N = len(datasets.keys())
price_means = []
price_stds = []
price_per_accommodated_means = []
price_per_accommodated_stds = []
cities = []

for city in datasets.keys():
    city_data = datasets[city]
    city_data = city_data[city_data['room_type'] == 'Private room']
    price_means.append(city_data['price'].mean())
    price_stds.append(city_data['price'].std())
    price_per_accommodated_means.append(city_data['price_per_accommodated'].mean())
    price_per_accommodated_stds.append(city_data['price_per_accommodated'].std())
    cities.append(city)


sort_indices = np.argsort(np.asarray(price_per_accommodated_means)) #sort cities on price per accommodated
price_means = list(np.asarray(price_means)[sort_indices[::-1]])
price_stds = list(np.asarray(price_stds)[sort_indices[::-1]])
price_per_accommodated_means = list(np.asarray(price_per_accommodated_means)[sort_indices[::-1]])
price_per_accommodated_stds = list(np.asarray(price_per_accommodated_stds)[sort_indices[::-1]])
cities = list(np.asarray(cities)[sort_indices[::-1]])


pos = np.arange(N)
width = 0.35

fig, ax = plt.subplots(figsize=(20,3))

prices_bar = ax.bar(pos, tuple(price_means), width, color='b', yerr=tuple(price_stds))
prices_pa_bar = ax.bar(pos+width, tuple(price_per_accommodated_means), width, 
                       color='g', yerr=tuple(price_per_accommodated_stds))

ax.legend((prices_bar[0], prices_pa_bar[0]), ('Price', 'Price per Accommodated'))

ax.set_ylabel('Dollars($)')
ax.set_title('Average Price and price per person for \'private room\' room type, per city')
ax.set_xticks(pos + width / 2)
ax.set_xticklabels(tuple(cities))

plt.show()


In [ ]:
#creating bar chart showing the average prices and average prices per accommodated for all room types, per city
plt.clf()
N = len(datasets.keys())
price_means = []
price_stds = []
price_per_accommodated_means = []
price_per_accommodated_stds = []
cities = []

for city in datasets.keys():
    city_data = datasets[city]
    city_data = city_data[city_data['room_type'] == 'Shared room']
    price_means.append(city_data['price'].mean())
    price_stds.append(city_data['price'].std())
    price_per_accommodated_means.append(city_data['price_per_accommodated'].mean())
    price_per_accommodated_stds.append(city_data['price_per_accommodated'].std())
    cities.append(city)


sort_indices = np.argsort(np.asarray(price_per_accommodated_means)) #sort cities on price per accommodated
price_means = list(np.asarray(price_means)[sort_indices[::-1]])
price_stds = list(np.asarray(price_stds)[sort_indices[::-1]])
price_per_accommodated_means = list(np.asarray(price_per_accommodated_means)[sort_indices[::-1]])
price_per_accommodated_stds = list(np.asarray(price_per_accommodated_stds)[sort_indices[::-1]])
cities = list(np.asarray(cities)[sort_indices[::-1]])


pos = np.arange(N)
width = 0.35

fig, ax = plt.subplots(figsize=(20,3))

prices_bar = ax.bar(pos, tuple(price_means), width, color='b', yerr=tuple(price_stds))
prices_pa_bar = ax.bar(pos+width, tuple(price_per_accommodated_means), width, 
                       color='g', yerr=tuple(price_per_accommodated_stds))

ax.legend((prices_bar[0], prices_pa_bar[0]), ('Price', 'Price per Accommodated'))

ax.set_ylabel('Dollars($)')
ax.set_title('Average Price and price per person for \'shared room\' room type, per city')
ax.set_xticks(pos + width / 2)
ax.set_xticklabels(tuple(cities))

plt.show()

In [ ]:
datasets['madrid']

In [ ]:
#convert last modified column to datetime
for city in datasets.keys():
    city_data = datasets[city]
    city_data['last_modified'] = pd.to_datetime(city_data['last_modified'])
    city_data = city_data.sort_values(by='last_modified',ascending=True)
    datasets[city] = city_data.reset_index()
    

In [ ]:
datasets['madrid'].head(5000)

In [ ]:
#obtain the room_ids with most frequencies per city and per room_type
N = 7 #number of most frequent room_ids we are going to use
room_types = data.groupby(['room_type']).groups.keys() #get all room_types
city_room_data = [] #containing the lists of most frequent rooms_ids per room_type per city
for city in cities: #used insead of simple dict iteration so that it is guaranteed that we always have the same order
    data = datasets[city]
    city_room = [] #containing the lists of most frequent room ids per room type
    for rt in room_types:
        gb = data[data['room_type'] == rt].groupby(['room_id']) #group by rooms_ids of a particular room_type
        city_room.append(list(gb['room_id'].count().sort_values(ascending=False).to_frame().index[:N].values.tolist()))
        #getting the first three room_ids with the biggest frequency per room_type
    city_room_data.append(city_room)
        

In [ ]:
#plotting the price changes per time for the most frequent room_ids per city through time
plt.clf()
fig, ax = plt.subplots(len(datasets.keys()),len(room_types), figsize=(20,60))
i = 0
for city in cities:
    data = datasets[city]
    data = data.set_index('last_modified')
    for j in range(len(room_types)):
        for rid in city_room_data[i][j]: #for each room id of ith city and jth room type
            data[data['room_id'] == rid]['price'].plot(ax=ax[i][j])
            ax[i][j].xaxis.label.set_visible(False)
    
    i += 1
[ax[i][0].set_ylabel(cities[i]) for i in range(len(cities))]
[ax[0][j].set_title(room_types[j]) for j in range(len(room_types))]
plt.show()

In [ ]:
#plotting the price_per_accommodated changes per time for the most frequent room_ids per room_type per city through time
plt.clf()
fig, ax = plt.subplots(len(datasets.keys()),len(room_types), figsize=(20,60))
i = 0
for city in cities:
    data = datasets[city]
    data = data.set_index('last_modified')
    for j in range(len(room_types)):
        for rid in city_room_data[i][j]: #for each room id of ith city and jth room type
            data[data['room_id'] == rid]['price_per_accommodated'].plot(ax=ax[i][j])
            ax[i][j].xaxis.label.set_visible(False)
    
    i += 1
[ax[i][0].set_ylabel(cities[i]) for i in range(len(cities))]
[ax[0][j].set_title(room_types[j]) for j in range(len(room_types))]
plt.show()

In [ ]:
plt.clf()
data = datasets['madrid']
data = data.set_index('last_modified')

data1 = data[data['room_type'] == 'Entire home/apt'][['price','price_per_accommodated']].resample('A').mean()
data2 = data[data['room_type'] == 'Entire home/apt'][['price','price_per_accommodated']].resample('A').std()
fig,ax = subplots()


data1.plot.bar(yerr=data2,ax=ax)
#data[data['reviews'] > 2].resample('A').mean().head(5)
#data[data['room_type'] == 'Entire home/apt'].resample('A').std()['price'].plot(kind='bar',err)
#xticks = []
#for ind in list(data1.index):
#    xticks.append(str(ind).split('-')[0])
#ax.set_xticklabels(tuple(xticks))
ax.set_xticklabels(tuple([str(ind).split('-')[0] for ind in list(data1.index)]))

#plt.show()

In [ ]:
#plotting the price and price_per_accommodated changes per time for the most frequent room_ids per room_type per city through time
plt.clf()
fig, ax = plt.subplots(len(datasets.keys()),len(room_types), figsize=(20,60))
i = 0
for city in cities:
    data = datasets[city]
    data = data.set_index('last_modified')
    for j in range(len(room_types)):
        means = data[data['room_type'] == room_types[j]][['price','price_per_accommodated']].resample('A').mean()
        stds = data[data['room_type'] == room_types[j]][['price','price_per_accommodated']].resample('A').std()
        means.dropna().plot.bar(yerr=stds,ax=ax[i][j])
        ax[i][j].set_xticklabels(tuple([str(label).split('-')[0] for label in list(means.index)]))
        ax[i][j].xaxis.label.set_visible(False)
        
    
    i += 1
[ax[i][0].set_ylabel(cities[i]) for i in range(len(cities))]
[ax[0][j].set_title(room_types[j]) for j in range(len(room_types))]
plt.show()
        

In [ ]:
#which percentage of the rooms of each city has above 2 reviews
for city,data in datasets.iteritems():
    print('{}% of {}\'s rooms have above 2 reviews'.format
          (100 * float(len(data[data['reviews'] > 2].index)/len(data.index)),city))

In [ ]:
data = datasets['madrid']
#print(data.groupby(['neighborhood']).groups)
#print(len(data.groupby(['borough']).groups))

In [ ]:
#checking the diversity in the nominal columns
#wherever there is only one value the column is deleted 
nominal_position_columns = ['neighborhood','borough']

for city in datasets.keys():
    data = datasets[city]
    for col in nominal_position_columns:
        if col in set(data.columns):
            num = len(data.groupby([col]).groups)
            print('\n{}: There are {} different {}s'.format(city,num,col))
            if num < 2:
                data.drop([col], axis=1, inplace=True)
                print('Column {} was deleted from {}\'s dataset'.format(col,city))
    datasets[city] = data
                    
                

In [ ]:
#finding out the percentage of the rooms that are presented to have 0 bedrooms
for city,data in datasets.iteritems():
    print('{}% of {}\'s rooms are presented to have zero bedrooms'.format(
        100 * len(data[data['bedrooms'] == 0]) / len(data.index),city))

In [ ]:
#mapping latitude and longitude to three dimensions(sphere) so that they can be standardized
#also transforming datetime to timestamp
new_datasets = dict() #containing he new data which only contain the 
for city in datasets.keys():
    data = datasets[city]
    new_data = data
    new_data['x'] = np.cos(new_data['latitude']) * np.cos(new_data['longitude'])
    new_data['y'] = np.cos(new_data['latitude']) * np.sin(new_data['longitude'])
    new_data['z'] = np.sin(new_data['latitude'])
    new_data = new_data.drop(['latitude','longitude'],axis=1) #deleting former coordinates
    for col in nominal_position_columns:
        if col in set(new_data.columns):
            new_data = new_data.drop([col],axis=1) #deleting nominal position features
    print('{}\'s coordinates system was transformed'.format(city))
    new_datasets[city] = new_data #inserted into new data dictionary

In [ ]:
#converting datetimes to timedeltas so that they can be standardized
for city in new_datasets.keys():
    data = new_datasets[city]
    data['last_modified'] = pd.to_timedelta(
    data['last_modified']).apply(lambda x: str(x).split()[0])
    print('{}\'s datetimes were converted to timedeltas'.format(city))


In [ ]:
#dropping room_id,host_id and other unused for the model to construct columns
#moreover room_type columns gets one-hot encoded
columns_to_drop = ['level_0','index','room_id','host_id','price_per_accommodated']
columns_to_onehot = ['room_type']

for city in new_datasets.keys():
    data = new_datasets[city]
    for col in columns_to_drop: #drop columns_to_drop
        if col in set(data.columns):
            data = data.drop([col],axis=1)
    for col in columns_to_onehot: #replace columns_to_onehot with their binary encoding
        if col in set(data.columns):
            one_hot_col = pd.get_dummies(data[col]) 
            data = data.join(one_hot_col)
            data = data.drop([col],axis=1) 
    print('Column deletes and replacements were completed for {}\'s dataset'.format(city))
    new_datasets[city] = data
    


In [ ]:
#***********************deleting rooms with bedrooms = 0
#no_bedrooms_datasets = dict()
#for city in new_datasets:
#    data = new_datasets[city]
#    no_bedrooms_datasets[city] = data.drop(data[data['bedrooms'] == 0].index)
    

In [ ]:
#transfering price to separate unit-column dataframe
prices = dict() #containing the price column (that is going to be predicted) for each city
for city in new_datasets.keys():
    data = new_datasets[city]
    price_data = pd.DataFrame()
    price_data['price'] = data['price']
    data = data.drop(['price'],axis=1)
    prices[city] = price_data
    new_datasets[city] = data
    


In [ ]:
prices['madrid']

In [ ]:
new_datasets['reykjavik']

In [ ]:
#splitting each cities dataset to training and test set
training_datasets = dict() #containing the training sets of each city
training_prices = dict() #containing the targets of the training set of each city
test_datasets = dict() #containing the test sets of each city
test_prices = dict() #containing the targets of the test set of each city

for city in new_datasets.keys():
    X = new_datasets[city]
    y = prices[city]
    X_train, X_test, y_train, y_test = train_test_split(X, y,test_size=0.2, random_state=random_state)
    training_datasets[city] = X_train
    training_prices[city] = y_train
    test_datasets[city] = X_test
    test_prices[city] = y_test
    print('{}:\tInitial size:\t{}\tTraining set size:\t{}\tTest set size:\t{}'.format(city,len(X.index),len(X_train.index),
                                                                                len(X_test.index)))




In [ ]:
training_datasets['helsinki']

In [ ]:
#Initializing 3-fold cross-validation
cv = KFold(n_splits=3, shuffle=True, random_state=random_state)

In [ ]:
#generating 3 training-validation set pairs from each city's training set
X_trains = dict() #containing the list of the training sets per city
X_vals = dict() #containing the list of the validation sets per city
y_trains = dict() #containing the list of the training sets' targets per city
y_vals = dict() #containing the list of the validation sets' targets per city

for city in training_datasets.keys():
    X = training_datasets[city]
    y = training_prices[city]
    X_trs = [] #the list of the training sets per city
    X_vs = [] #the list of the validation sets per city
    y_trs = [] #the list of the training sets' targets per city
    y_vs = [] #the list of the validation sets' targets per city
    for train_index, val_index in cv.split(X): #splitting to training and validation set
        X_trs.append(X.iloc[train_index])
        y_trs.append(y.iloc[train_index])
        X_vs.append(X.iloc[val_index])
        y_vs.append(y.iloc[val_index])
    X_trains[city] = X_trs
    X_vals[city] = X_vs
    y_trains[city] = y_trs
    y_vals[city] = y_vs
    print('{}:\tInitial size of training data:\t{}\tTraining sets\' size:\t{}\tValidation sets\' size:\t{}'.format(city,len(X.index),len(X_trs[0].index),
                                                                                len(X_vs[0].index)))


In [ ]:
y_trains['paris'][0].head(5)

In [ ]:
#standardizing the non-binary features as well as the targets(price)

for city in X_trains.keys():
    X_trs = X_trains[city]
    X_vs = X_vals[city]
    y_trs = y_trains[city]
    y_vs = y_vals[city]
    X_columns = list(X_trs[0]) #holdinh the column names of the dataset
    y_column = list(y_trs[0]) #holding the column name of the targets
    X_trs_sc = [] #they are going to host the standardized versions
    X_vs_sc = []
    y_trs_sc = []
    y_vs_sc = []
    stand_columns = [col for col in X_columns if col not in set(room_types)] 
    #holding the names of the columns to be standardized
        
    
    for i in range(len(X_trs)):
        X_tr_index = list(X_trs[i].index) #holding the index of the particular cross-validation split
        X_v_index = list(X_vs[i].index) #holding the index of the particular cross-validation split
        X_tr_stand = X_trs[i][stand_columns] #holding the features to be standardized
        X_v_stand = X_vs[i][stand_columns] #holding the features to be standardized
        X_tr = X_trs[i].drop(stand_columns,axis=1) #holding only the binary columns
        X_v = X_vs[i].drop(stand_columns,axis=1) #holding only the binary columns
        non_stand_columns = list(X_tr.columns) #holding the names of the binary columns
        X_sc = StandardScaler().fit(X_tr_stand) 
        X_tr_stand = X_sc.transform(X_tr_stand) #standardization of the training set
        X_v_stand = X_sc.transform(X_v_stand) #standardization of the validation set using the model trained on 
                                                    #the training set
        X_tr = np.c_[X_tr_stand,X_tr] #uniting standardized features with binary features
        X_v = np.c_[X_v_stand,X_v] #uniting standardized features with binary features
            
        X_trs_sc.append(pd.DataFrame(X_tr,columns=stand_columns+non_stand_columns,index=X_tr_index))
        X_vs_sc.append(pd.DataFrame(X_v,columns=stand_columns+non_stand_columns,index=X_v_index))
        
        y_tr = y_trs[i].values
        y_v = y_vs[i].values
        y_sc = StandardScaler().fit(y_tr) 
        y_tr = y_sc.transform(y_tr) #standardization of the training targets
        y_v = y_sc.transform(y_v) #standardization of the validation targets using training targets' model
        y_trs_sc.append(pd.DataFrame(y_tr,columns=y_column,index=X_tr_index))
        y_vs_sc.append(pd.DataFrame(y_v,columns=y_column,index=X_v_index))
    
    
    X_trains[city] = X_trs_sc
    X_vals[city] = X_vs_sc
    y_trains[city] = y_trs_sc
    y_vals[city] = y_vs_sc
    
    print('{}\'s data were standardized'.format(city))
y_trains['paris'][0].head(5)
        
        

In [ ]:
l = []
a = np.array([1,2,3,4])
b = np.array([2,3,4,5])
l.append(a)
l.append(b)
l = np.vstack(l)
print(np.mean(l,axis=0))

In [ ]:
#Performing stability selection (trying to estimate the value of each ferature as far as the regression is concerned)
#input: X_trains_sc --> list of standardized training sets
#       y_trains_sc --> list of standardized training targets
#output: list containing the mean and standard deviation of the metric 
def get_features_importance(X_trains_sc,y_trains_sc,selection_threshold=0.25):
    rlasso = RandomizedLasso(random_state=random_state,selection_threshold=selection_threshold)
    cv_scores = []
    for i in range(len(X_trains_sc)):
        rlasso.fit(X_trains_sc[i],y_trains_sc[i].values.ravel())
        cv_scores.append(rlasso.scores_)
    cv_scores = np.asarray(cv_scores)
    cv_scores = np.vstack(cv_scores) #list of arrays to array
    return [list(np.mean(cv_scores,axis=0)),list(np.std(cv_scores,axis=0))]
    #returning a list containing the list containing the means and the stds of the metric per feature
    



In [ ]:
#Performing Linear Regression
#input: X_trains_sc --> list of standardized training sets
#       y_trains_sc --> list of standardized training targets
#       X_vals_sc --> list of standardized validation sets
#       y_vals_sc --> list of standardized validation targets
#output: list containing the means and standard deviations of regression evaluation metrics 
def get_linear_regression(X_trains_sc,y_trains_sc,X_vals_sc,y_vals_sc):
    lr = LinearRegression()
    cv_mses = []
    cv_maes = []
    cv_r2s = []
    for i in range(len(X_trains_sc)):
        lr.fit(X_trains_sc[i],y_trains_sc[i].values.ravel())
        y_pred = lr.predict(X_vals_sc[i])
        cv_mses.append(math.sqrt(mean_squared_error(y_vals_sc[i],y_pred)))
        cv_maes.append(mean_absolute_error(y_vals_sc[i],y_pred))
        cv_r2s.append(r2_score(y_vals_sc[i],y_pred))
    cv_mses = np.asarray(cv_mses)
    cv_maes = np.asarray(cv_maes)
    cv_r2s = np.asarray(cv_r2s)
    
    rmse = [np.mean(cv_mses),np.std(cv_mses)] #list containing the mean and the std of the particular metric
    mae = [np.mean(cv_maes),np.std(cv_maes)]
    r2 = [np.mean(cv_r2s),np.std(cv_r2s)]
    return [rmse,mae,r2] #returning the metrics

In [ ]:
#Performing Support Vector Machines Regression
#input: X_trains_sc --> list of standardized training sets
#       y_trains_sc --> list of standardized training targets
#       X_vals_sc --> list of standardized validation sets
#       y_vals_sc --> list of standardized validation targets
#output: list containing the means and standard deviations of regression evaluation metrics 
def get_svm_regression(X_trains_sc,y_trains_sc,X_vals_sc,y_vals_sc):
    clf = SVR()
    cv_mses = []
    cv_maes = []
    cv_r2s = []

    for i in range(len(X_trains_sc)):
        clf.fit(X_trains_sc[i],y_trains_sc[i].values.ravel())
        y_pred = clf.predict(X_vals_sc[i])
        cv_mses.append(math.sqrt(mean_squared_error(y_vals_sc[i],y_pred)))
        cv_maes.append(mean_absolute_error(y_vals_sc[i],y_pred))
        cv_r2s.append(r2_score(y_vals_sc[i],y_pred))
    cv_mses = np.asarray(cv_mses)
    cv_maes = np.asarray(cv_maes)
    cv_r2s = np.asarray(cv_r2s)
    rmse = [np.mean(cv_mses),np.std(cv_mses)] #list containing the mean and the std of the particular metric
    mae = [np.mean(cv_maes),np.std(cv_maes)]
    r2 = [np.mean(cv_r2s),np.std(cv_r2s)]
    return [rmse,mae,r2] #returning the metrics

In [ ]:
#Performing Multi Layer Perceptron Regression
#input: X_trains_sc --> list of standardized training sets
#       y_trains_sc --> list of standardized training targets
#       X_vals_sc --> list of standardized validation sets
#       y_vals_sc --> list of standardized validation targets
#output: list containing the means and standard deviations of regression evaluation metrics 
def get_mlp_regression(X_trains_sc,y_trains_sc,X_vals_sc,y_vals_sc):
    mlpr = MLPRegressor(random_state=random_state)
    cv_mses = []
    cv_maes = []
    cv_r2s = []
    for i in range(len(X_trains_sc)):
        mlpr.fit(X_trains_sc[i],y_trains_sc[i].values.ravel())
        y_pred = mlpr.predict(X_vals_sc[i])
        cv_mses.append(math.sqrt(mean_squared_error(y_vals_sc[i],y_pred)))
        cv_maes.append(mean_absolute_error(y_vals_sc[i],y_pred))
        cv_r2s.append(r2_score(y_vals_sc[i],y_pred))
    cv_mses = np.asarray(cv_mses)
    cv_maes = np.asarray(cv_maes)
    cv_r2s = np.asarray(cv_r2s)
    rmse = [np.mean(cv_mses),np.std(cv_mses)] #list containing the mean and the std of the particular metric
    mae = [np.mean(cv_maes),np.std(cv_maes)]
    r2 = [np.mean(cv_r2s),np.std(cv_r2s)]
    return [rmse,mae,r2] #returning the metrics

In [ ]:
#Performing Random Forest Regression
#input: X_trains_sc --> list of standardized training sets
#       y_trains_sc --> list of standardized training targets
#       X_vals_sc --> list of standardized validation sets
#       y_vals_sc --> list of standardized validation targets
#output: list containing the means and standard deviations of regression evaluation metrics 
def get_rf_regression(X_trains_sc,y_trains_sc,X_vals_sc,y_vals_sc):
    rfr = RandomForestRegressor(random_state=random_state)
    cv_mses = []
    cv_maes = []
    cv_r2s = []
    for i in range(len(X_trains_sc)):
        rfr.fit(X_trains_sc[i],y_trains_sc[i].values.ravel())
        y_pred = rfr.predict(X_vals_sc[i])
        cv_mses.append(math.sqrt(mean_squared_error(y_vals_sc[i],y_pred)))
        cv_maes.append(mean_absolute_error(y_vals_sc[i],y_pred))
        cv_r2s.append(r2_score(y_vals_sc[i],y_pred))
    cv_mses = np.asarray(cv_mses)
    cv_maes = np.asarray(cv_maes)
    cv_r2s = np.asarray(cv_r2s)
    rmse = [np.mean(cv_mses),np.std(cv_mses)] #list containing the mean and the std of the particular metric
    mae = [np.mean(cv_maes),np.std(cv_maes)]
    r2 = [np.mean(cv_r2s),np.std(cv_r2s)]
    return [rmse,mae,r2] #returning the metrics

In [ ]:
#trying to estimate the importance of the features with stability selection
selection_threshold=0.3 #the fraction of features selected in each sub-sample of the method
for city in X_trains.keys():
    print('{}:'.format(city))
    X_trs = X_trains[city]
    y_trs = y_trains[city]
    features = list(X_trs[0]) #holding the names of the features
    results = get_features_importance(X_trs,y_trs,selection_threshold)
    for i in range(len(results[0])):
        print('{0}: {1} +- {2:.4f}'.format(features[i],results[0][i],results[1][i]))
    print()
    
    
    
    


In [ ]:
#trying and evaluating different classifiers (using all columns)
print('\tRoot Mean Square Error\tMean Absolute Error\tR2 Score')
for city in X_trains.keys():
    print('\n{}:\n'.format(city))
    X_trs = X_trains[city]
    y_trs = y_trains[city]
    X_vs = X_vals[city]
    y_vs = y_vals[city]
    lr_res = get_linear_regression(X_trs,y_trs,X_vs,y_vs)
    print('LR',end='')
    for res in lr_res:
        print('\t{0:.4f} +- {1:.4f}'.format(res[0],res[1]),end='')
    print()
    #svm_res = get_svm_regression(X_trs,y_trs,X_vs,y_vs)
    #print('SVM',end='')
    #for res in svm_res:
        #print('\t{0:.4f} +- {1:.4f}'.format(res[0],res[1]),end='')
    #print()
    mlp_res = get_mlp_regression(X_trs,y_trs,X_vs,y_vs)
    print('MLP',end='')
    for res in mlp_res:
        print('\t{0:.4f} +- {1:.4f}'.format(res[0],res[1]),end='')
    print()
    rf_res = get_rf_regression(X_trs,y_trs,X_vs,y_vs)
    print('RF',end='')
    for res in rf_res:
        print('\t{0:.4f} +- {1:.4f}'.format(res[0],res[1]),end='')
    print()